In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import math
import statistics
from statistics import mean

In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor

# Pre-processing

## Training set

In [3]:
filename='./Data/train.csv'
df_train = pd.read_csv(filename)
# drop the rows including NaN
df_train.dropna(axis = 0, how = 'any', inplace = True)

#print (df_train[0:7])

### Compute interval of purchase date and release date(pri)

In [5]:
# transfer purchase_date to format
months=[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
        ['01' ,'02' ,'03' ,'04' ,'05' ,'06' ,'07' ,'08' ,'09' ,'10' ,'11' ,'12'] ]
index_d=0;
for d in df_train.purchase_date:
    # handling single number day
    if len(d) == 8:
        d = '0'+ d
    
    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[3:6], month)
            break
    
    df_train.purchase_date.iloc[index_d] = d
    index_d = index_d + 1
#    print ('d: '+(d))
#    d = datetime.datetime.strptime(d, date_format)
#    print (d)




02-07-18
26-11-16
02-07-18
28-11-16
04-03-18
16-02-18
29-01-18
23-11-18
22-12-17
30-01-18
23-06-17
20-07-19
09-10-16
13-06-18
16-09-17
13-01-18
30-05-19
18-02-18
21-12-18
30-01-18
28-11-16
16-05-18
19-11-17
02-04-19
09-05-18
09-12-17
27-06-19
21-12-18
02-10-17
10-12-17
18-03-18
28-02-18
02-07-18
23-10-18
17-03-17
08-12-17
27-06-19
28-11-17
22-11-16
22-11-18
27-06-19
29-03-18
28-11-16
28-10-18
21-03-18
24-03-19
28-10-17
16-02-18
16-11-17
21-09-17
28-06-17
16-04-18
14-12-17
29-09-17
05-10-17
28-10-17
08-04-18
22-11-18
10-02-19
19-11-17
13-03-18
21-12-17
03-08-15
16-02-18
20-06-19
28-10-17
19-11-17
16-09-18
28-06-17
21-06-18
27-02-19
23-04-18
01-03-18
11-04-17
30-01-18
02-07-18
27-06-19
01-08-17
27-06-19
08-02-19
03-11-16
23-02-19
22-12-17
04-01-17
20-09-16
30-01-18
29-07-19
27-11-17
07-12-17
27-06-19
11-03-18
11-05-18
28-11-16
27-12-17
17-11-18
09-11-16
16-02-18
11-05-18
31-10-17
25-10-17
03-03-17
28-10-17
10-03-17
08-03-18
09-04-19
01-09-18
27-02-18
14-04-19
09-11-16
22-03-18
21-11-17
2

In [6]:
# transfer purchase_date to date-type
date_format = '%d-%m-%y'
df_train['purchase_date'] = pd.to_datetime(df_train['purchase_date'],format=date_format)
df_train.purchase_date[0:6]

0   2018-07-02
1   2016-11-26
2   2018-07-02
3   2016-11-28
4   2018-03-04
6   2018-02-16
Name: purchase_date, dtype: datetime64[ns]

In [7]:
# transfer release_date to format
index_d=0;

for d in df_train.release_date:
    d=d.replace(' ', '-')
    d=d.replace(',', '')

    # handling singe number day
    if len(d) == 10:
        d = '0'+ d

    # handling year has only 2 digits
    d = d.split('-')
    if len(d[2])!=4:
        d[2] = '20'+d[2]
    d = d[0]+'-'+d[1]+'-'+d[2]

    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[3:6], month)
            break
    
    df_train.release_date.iloc[index_d] = d
    index_d = index_d + 1


In [8]:
# transfer release_date to date-type
date_format_r = '%d-%m-%Y'
df_train['release_date'] = pd.to_datetime(df_train['release_date'],format = date_format_r)
df_train.release_date

0     2013-12-10
1     2015-08-12
2     2014-01-28
3     2010-03-31
4     2012-07-30
         ...    
352   2016-12-20
353   2015-06-23
354   2015-11-05
355   2016-06-03
356   2013-12-05
Name: release_date, Length: 355, dtype: datetime64[ns]

In [52]:
# compute the interval of purchase_date and release_date

df_train['pur_rel_itv'] = df_train['purchase_date']-df_train['release_date']
df_train = df_train.reset_index()
df_train['pur_rel_itv'][0:6]

0   1665 days
1    472 days
2   1616 days
3   2434 days
4   2043 days
5   1942 days
Name: pur_rel_itv, dtype: timedelta64[ns]

In [54]:
# convert pur_rel_itv type to integer
index_pri = 0
for p in df_train.pur_rel_itv:
#    print (p)
    df_train.pur_rel_itv[index_pri] = p.days
    index_pri = index_pri + 1



#df_train['pur_rel_itv'] = dfp_box

In [90]:
#df_train.pur_rel_itv

df_train1

,index,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665
1,1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,472
2,2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,1616
3,3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,2434
4,4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,2043
...,...,...,...,...,...,...,...,...,...,...,...,...
350,352,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,339
351,353,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,1149
352,354,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,817
353,355,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,477


In [88]:
df_train = df_train.drop(columns = ['id'])

KeyError: "['id'] not found in axis"

In [98]:
# drop rows whose pri < 0
df_train1 = df_train[df_train['pur_rel_itv'] >= 0]


In [99]:
df_train1 = df_train1.reset_index(drop=True)
df_train1 = df_train1.drop(columns=['index'])

In [100]:
df_train1

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,472
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,1616
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,2434
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,2043
...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,339
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,1149
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,817
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,477


### One-hot genres

In [101]:
# get all kinds of genres
genres_box_tmp=[]
for g in df_train1.genres:
    g = g.replace(' ', '')
    genres_box_tmp.extend(re.split(r",|&", g))# split string with "," and "&"

genres_box = list(set(genres_box_tmp))# get rid of redundent element
genres_box.sort()
genres_box

['Action',
 'Adventure',
 'Animation',
 'AudioProduction',
 'Casual',
 'Design',
 'EarlyAccess',
 'FreetoPlay',
 'Gore',
 'Illustration',
 'Indie',
 'MassivelyMultiplayer',
 'Modeling',
 'Nudity',
 'RPG',
 'Racing',
 'SexualContent',
 'Simulation',
 'Sports',
 'Strategy',
 'Utilities',
 'Violent']

In [102]:
print (df_train1.shape)

(340, 11)


In [103]:
dfgres = pd.DataFrame(columns = genres_box, index = range(len(df_train1)))

In [104]:
count = 0
for g in df_train1.genres:
#    print (count, g)
    g = g.replace(' ', '')
    g = re.split(r",|&", g)
    
    for g_i in g:
        if g_i in genres_box:
            dfgres[g_i].iloc[count] = 1
#        print (count, g_i)
    count = count + 1



In [105]:
dfgres = dfgres.fillna(0)

In [106]:
df_train2 = pd.concat([df_train1,dfgres], axis = 1, sort = False)

In [107]:
df_train2

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,...,0,0,1,0,0,0,0,0,0,0
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,...,0,0,0,0,0,1,0,0,0,0
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,...,0,0,0,0,0,0,0,0,0,0
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,...,0,0,1,0,0,0,0,0,0,0
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
df_train2.to_csv("train_prepro_1117.csv",sep = ',',index = False)

## Testing set 

In [108]:
filename='./Data/test.csv'
df_test = pd.read_csv(filename)

In [110]:
# handling empty value in total_positive_reviews and total_negative_reviews

df_test = df_test.fillna({'total_positive_reviews': df_test['total_positive_reviews'].mean(),'total_negative_reviews': df_test['total_negative_reviews'].mean()})

In [112]:
# handling empty value in purchase_date or release_date
df_test = df_test.fillna(method='ffill')

In [114]:
# date transform in release_date

index_d=0;
for d in df_test.release_date:
    # handling singe number day
    if len(d) == 8:
        d = '0'+ d
    
    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
#    print (month)
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[3:6], month)
            break
#    print (d)
    df_test.release_date.iloc[index_d] = d
    index_d = index_d + 1

In [115]:
# transfer release_date to date-type
date_format = '%d-%m-%y'
df_test['release_date'] = pd.to_datetime(df_test['release_date'],format = date_format)
df_test.release_date[0:6]

0   2012-05-22
1   2018-04-24
2   2017-03-20
3   2007-07-17
4   2013-08-22
5   2018-02-28
Name: release_date, dtype: datetime64[ns]

In [116]:
# date transform in purchase_date
index_d=0;

for d in df_test.purchase_date:
    d = d.replace(' ', '-')
    d = d.replace(',', '')
    
    # converting month into digital
    month=''.join(re.findall(r'[A-Za-z]', d))
    for m in range(len(months[0][:])):
        if month == months[0][m]:
            month = months[1][m]
            d = d.replace(d[0:3], month)
            break
            
    df_test.purchase_date.iloc[index_d] = d
    index_d = index_d + 1
#    print (d)

In [117]:
# transfer purchase_date to date-type
date_format_r = '%m-%d-%Y'
df_test['purchase_date'] = pd.to_datetime(df_test['purchase_date'],format = date_format_r)
df_test.purchase_date

0    2018-10-28
1    2019-07-20
2    2019-07-16
3    2018-01-30
4    2017-02-24
        ...    
85   2018-03-23
86   2018-03-03
87   2018-02-18
88   2017-11-19
89   2018-11-25
Name: purchase_date, Length: 90, dtype: datetime64[ns]

### compute pri

In [119]:
# compute the interval of purchase_date and release_date

df_test['pur_rel_itv'] = df_test['purchase_date']-df_test['release_date']

index_pri = 0
for p in df_test.pur_rel_itv:
#    print (p)
    df_test.pur_rel_itv[index_pri] = p.days
    index_pri = index_pri + 1


In [120]:
# if pri < 0, replace with mean
index_p = 0
for pri in df_test.pur_rel_itv:
    if pri < 0:
        df_test.pur_rel_itv.iloc[index_p] = df_test['pur_rel_itv'].mean()
    index_p = index_p + 1

In [121]:
df_test

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,2350
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,452
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,848
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,3850
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,1282
...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,1766
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,3092
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,1650
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,153


### One-hot genres

In [123]:
#genres_box

dfgres2 = pd.DataFrame(columns = genres_box, index = range(len(df_test)))
count = 0
for g in df_test.genres:
#    print (count, g)
    g = g.replace(' ', '')
    g = re.split(r",|&", g)
    
    for g_i in g:
        if g_i in genres_box:
            dfgres2[g_i].iloc[count] = 1
#        print (count, g_i)
    count = count + 1

In [124]:
dfgres2 = dfgres2.fillna(0)
dfgres2

,Action,Adventure,Animation,AudioProduction,Casual,Design,EarlyAccess,FreetoPlay,Gore,Illustration,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
88,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,0,0


In [125]:
df_test2 = pd.concat([df_test,dfgres2], axis = 1, sort = False)
df_test2

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,...,0,0,0,0,0,0,0,0,0,0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,...,0,0,0,0,0,0,0,1,0,0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,...,0,0,0,0,0,1,0,1,0,0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,...,0,0,0,0,0,0,0,1,0,0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,...,0,0,0,0,0,0,0,0,0,0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,...,0,0,0,0,0,0,0,0,0,0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,...,0,0,0,0,0,1,0,1,0,0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,...,0,0,1,0,0,1,1,1,0,0


In [189]:
df_test2.to_csv("test_prepro_1117.csv",sep = ',',index = False)

# Feature selection

In [133]:
scaler = RobustScaler()

X = df_train2.drop(columns=['playtime_forever', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
test_X = df_test2.drop(columns=['id', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
y= df_train2.playtime_forever

X_scaled = scaler.fit(X).transform(X)
y_log = np.log(df_train2.playtime_forever + 1)
test_X_scaled = scaler.transform(test_X)

In [135]:
X_scaled

array([[-0.59047619, -0.28530217, -0.30156538, ...,  0.        ,
         0.        ,  0.        ],
       [-1.2952381 , -0.32639604, -0.38996317, ...,  0.        ,
         0.        ,  0.        ],
       [-0.34285714,  0.02625769,  0.22053407, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.28571429,  0.27129022,  1.19290976, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.24456154, -0.24355433, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60952381, -0.22136528, -0.29604052, ...,  1.        ,
         0.        ,  0.        ]])

# Modeling & Evaluation 

In [136]:
# define cross validation stragegy RMSE
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [137]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(),SGDRegressor(),BayesianRidge(),KernelRidge(),
          ExtraTreesRegressor(),XGBRegressor()]

In [139]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 0.869213, 0.1263
Ridge: 0.856425, 0.1207
Lasso: 0.859810, 0.1374
RF: 0.828853, 0.1114
GBR: 0.872621, 0.1249
SVR: 0.842413, 0.1377
LinSVR: 0.981087, 0.1116
Ela: 0.859178, 0.1437
SGD: 290869196007393.125000, 353497209475596.1875
Bay: 0.842490, 0.1292
Ker: 0.884607, 0.1286
Extra: 0.840198, 0.1408
[10:52:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:52:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Xgb: 0.836737, 0.1187


## Fine Tuning Parameters

In [140]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

In [146]:
# Ridge - USABLE 

grid(Ridge()).grid_get(X_scaled,y_log,{'alpha':[37.1,37.2,37.3,37.4,37.5,37.6,37.7,37.8,37.9,37,38]})

{'alpha': 37.5} 0.851126303602662
             params  mean_test_score  std_test_score
0   {'alpha': 37.1}         0.851127        0.228055
1   {'alpha': 37.2}         0.851126        0.228069
2   {'alpha': 37.3}         0.851126        0.228083
3   {'alpha': 37.4}         0.851126        0.228097
4   {'alpha': 37.5}         0.851126        0.228111
5   {'alpha': 37.6}         0.851126        0.228125
6   {'alpha': 37.7}         0.851126        0.228138
7   {'alpha': 37.8}         0.851127        0.228152
8   {'alpha': 37.9}         0.851127        0.228166
9     {'alpha': 37}         0.851127        0.228041
10    {'alpha': 38}         0.851127        0.228180


In [161]:
# RF - USABLE

grid(RandomForestRegressor()).grid_get(X_scaled,y_log,{'max_features': ["auto","sqrt","log2"]})

{'max_features': 'sqrt'} 0.7979814735601171
                     params  mean_test_score  std_test_score
0  {'max_features': 'auto'}         0.853714        0.286907
1  {'max_features': 'sqrt'}         0.797981        0.233246
2  {'max_features': 'log2'}         0.829482        0.278306


In [149]:
# Lasso

grid(Lasso()).grid_get(X_scaled,y_log,{'alpha': [0.001,0.005,0.01,0.015,0.02,0.05,0.1],'max_iter':[10000]})

{'alpha': 0.015, 'max_iter': 10000} 0.8487055707444151
                                params  mean_test_score  std_test_score
0  {'alpha': 0.001, 'max_iter': 10000}         0.869101        0.224891
1  {'alpha': 0.005, 'max_iter': 10000}         0.856353        0.218249
2   {'alpha': 0.01, 'max_iter': 10000}         0.849912        0.218573
3  {'alpha': 0.015, 'max_iter': 10000}         0.848706        0.218463
4   {'alpha': 0.02, 'max_iter': 10000}         0.848790        0.220182
5   {'alpha': 0.05, 'max_iter': 10000}         0.857456        0.237627
6    {'alpha': 0.1, 'max_iter': 10000}         0.867640        0.246853


In [151]:
# SVR - USABLE

grid(SVR()).grid_get(X_scaled,y_log,{'C':[0.5,1,1.5],'kernel':["linear","poly","rbf"],'degree':[2,3,4]})

{'C': 1, 'degree': 2, 'kernel': 'rbf'} 0.8535943732172607
                                         params  mean_test_score  \
0   {'C': 0.5, 'degree': 2, 'kernel': 'linear'}     9.368143e-01   
1     {'C': 0.5, 'degree': 2, 'kernel': 'poly'}     9.538928e+01   
2      {'C': 0.5, 'degree': 2, 'kernel': 'rbf'}     8.586730e-01   
3   {'C': 0.5, 'degree': 3, 'kernel': 'linear'}     9.368143e-01   
4     {'C': 0.5, 'degree': 3, 'kernel': 'poly'}     4.207835e+04   
5      {'C': 0.5, 'degree': 3, 'kernel': 'rbf'}     8.586730e-01   
6   {'C': 0.5, 'degree': 4, 'kernel': 'linear'}     9.368143e-01   
7     {'C': 0.5, 'degree': 4, 'kernel': 'poly'}     1.202207e+07   
8      {'C': 0.5, 'degree': 4, 'kernel': 'rbf'}     8.586730e-01   
9     {'C': 1, 'degree': 2, 'kernel': 'linear'}     9.387332e-01   
10      {'C': 1, 'degree': 2, 'kernel': 'poly'}     9.000523e+01   
11       {'C': 1, 'degree': 2, 'kernel': 'rbf'}     8.535944e-01   
12    {'C': 1, 'degree': 3, 'kernel': 'linear'}     9.3873

In [156]:
# Ela

grid(ElasticNet()).grid_get(X_scaled,y_log,{'alpha':[0.031,0.032,0.033,0.034,0.035,0.036,0.037,0.038,0.039,0.03]})

{'alpha': 0.031} 0.8485073772942211
             params  mean_test_score  std_test_score
0  {'alpha': 0.031}         0.848507        0.220380
1  {'alpha': 0.032}         0.848509        0.220533
2  {'alpha': 0.033}         0.848523        0.220679
3  {'alpha': 0.034}         0.848583        0.220889
4  {'alpha': 0.035}         0.848615        0.221133
5  {'alpha': 0.036}         0.848654        0.221375
6  {'alpha': 0.037}         0.848697        0.221616
7  {'alpha': 0.038}         0.848746        0.221855
8  {'alpha': 0.039}         0.848801        0.222092
9   {'alpha': 0.03}         0.848513        0.220229


In [157]:
# Bay - USABLE

grid(BayesianRidge()).grid_get(X_scaled,y_log,{})

{} 0.8523399080970574
  params  mean_test_score  std_test_score
0     {}          0.85234        0.228766


In [164]:
# Extra 

grid(ExtraTreesRegressor()).grid_get(X_scaled,y_log,{'max_features':["auto","sqrt","log2"],'bootstrap':[True]})

{'bootstrap': True, 'max_features': 'auto'} 0.8348347892667347
                                        params  mean_test_score  \
0  {'bootstrap': True, 'max_features': 'auto'}         0.834835   
1  {'bootstrap': True, 'max_features': 'sqrt'}         0.846711   
2  {'bootstrap': True, 'max_features': 'log2'}         0.852657   

   std_test_score  
0        0.221662  
1        0.256283  
2        0.260787  


In [159]:
# Xgb - USABLE 

grid(XGBRegressor()).grid_get(X_scaled,y_log,{'booster':["gbtree", "dart"],'tree_method':["auto", "exact", "approx", "hist"],'verbosity':[0],'eta':[0.001,0.01,0.1,0.5]})

{'booster': 'dart', 'eta': 0.001, 'tree_method': 'auto', 'verbosity': 0} 0.8451179286421797
                                               params  mean_test_score  \
0   {'booster': 'gbtree', 'eta': 0.001, 'tree_meth...         0.845118   
1   {'booster': 'gbtree', 'eta': 0.001, 'tree_meth...         0.845118   
2   {'booster': 'gbtree', 'eta': 0.001, 'tree_meth...         0.869342   
3   {'booster': 'gbtree', 'eta': 0.001, 'tree_meth...         0.851393   
4   {'booster': 'gbtree', 'eta': 0.01, 'tree_metho...         0.845118   
5   {'booster': 'gbtree', 'eta': 0.01, 'tree_metho...         0.845118   
6   {'booster': 'gbtree', 'eta': 0.01, 'tree_metho...         0.869342   
7   {'booster': 'gbtree', 'eta': 0.01, 'tree_metho...         0.851393   
8   {'booster': 'gbtree', 'eta': 0.1, 'tree_method...         0.845118   
9   {'booster': 'gbtree', 'eta': 0.1, 'tree_method...         0.845118   
10  {'booster': 'gbtree', 'eta': 0.1, 'tree_method...         0.869342   
11  {'booster': 'gbt

# Ensemble Methods

In [160]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [166]:
# sorted by performance(good -> bad)
rf = RandomForestRegressor(max_features = 'sqrt')
extra = ExtraTreesRegressor(bootstrap = True, max_features = 'auto')
xgb = XGBRegressor(booster = 'dart', eta = 0.001, tree_method = 'auto', verbosity = 0)
ridge = Ridge(alpha = 37.5)
bay = BayesianRidge()
svr = SVR(C = 1, degree = 2, kernel = 'rbf')

In [167]:
# assign weights based on their gridsearch score

w1 = 0.25
w2 = 0.2
w3 = 0.2
w4 = 0.15
w5 = 0.15
w6 = 0.05

In [168]:
weight_avg = AverageWeight(mod = [rf,extra,xgb,ridge,bay,svr],weight=[w1,w2,w3,w4,w5,w6])

In [170]:
rmse_cv(weight_avg,X_scaled,y_log),  rmse_cv(weight_avg,X_scaled,y_log).mean()

(array([0.7175688 , 0.68803973, 0.9062082 , 0.98602316, 0.62809015]),
 0.7852918732816356)

In [172]:
# if using two best-performed model - not as good as six combined
weight_avg = AverageWeight(mod = [rf,extra],weight=[0.5,0.5])
rmse_cv(weight_avg,X_scaled,y_log),  rmse_cv(weight_avg,X_scaled,y_log).mean()

(array([0.73796013, 0.74168008, 0.92307829, 1.03548863, 0.68260356]),
 0.8045106109433983)

# Stacking

In [173]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [174]:
# must do imputer first, otherwise stacking won't work, and i don't know why.
a = Imputer().fit_transform(X_scaled)
b = Imputer().fit_transform(y_log.values.reshape(-1,1)).ravel()

C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [175]:
stack_model = stacking(mod = [rf,extra,xgb,ridge,bay,svr], meta_model = rf)

In [176]:
print(rmse_cv(stack_model,a,b))
print(rmse_cv(stack_model,a,b).mean())

[0.70491578 0.73851551 1.04929122 1.04734301 0.7468221 ]
0.8247039295928367


### Extract the features generated from stacking, then combine them with original features.

In [177]:
X_train_stack, X_test_stack = stack_model.get_oof(a,b,test_X_scaled)

In [178]:
X_train_stack.shape, a.shape

((340, 6), (340, 26))

In [179]:
X_train_add = np.hstack((a,X_train_stack))

In [180]:
X_test_add = np.hstack((test_X_scaled,X_test_stack))

In [181]:
X_train_add.shape, X_test_add.shape

((340, 32), (90, 32))

In [182]:
print(rmse_cv(stack_model,X_train_add,b))
print(rmse_cv(stack_model,X_train_add,b).mean())

[0.76222964 0.68238968 0.9057833  1.00684369 0.66357372]
0.8319341712109608


# Submission

In [183]:
# Final Model

stack_model = stacking(mod = [rf,extra,xgb,ridge,bay,svr], meta_model = rf)

In [184]:
stack_model.fit(a,b)

stacking(meta_model=RandomForestRegressor(bootstrap=True, criterion='mse',
                                          max_depth=None, max_features='sqrt',
                                          max_leaf_nodes=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=10, n_jobs=None,
                                          oob_score=False, random_state=None,
                                          verbose=0, warm_start=False),
         mod=[RandomFor...
                    normalize=False, random_state=None, solver='auto',
                    tol=0.001),
              BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False,
                   

In [185]:
pred = np.exp(stack_model.predict(test_X_scaled) + 1)

In [187]:
result=pd.DataFrame({'id':range(len(df_test2)), 'playtime_forever':pred})
result.to_csv("submission.csv",index=False)